# NBA  Project

### Import packages and begin evaluating endpoints

In [21]:
import pandas as pd

In [22]:
from nba_api.stats.endpoints import teamyearbyyearstats

In [23]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()

In [24]:
wiz = [team for team in nba_teams
         if team['full_name'] == 'Washington Wizards'][0]
wiz

{'id': 1610612764,
 'full_name': 'Washington Wizards',
 'abbreviation': 'WAS',
 'nickname': 'Wizards',
 'city': 'Washington',
 'state': 'District of Columbia',
 'year_founded': 1961}

In [25]:
wiz_id = '1610612764'
wiz_stats = teamyearbyyearstats.TeamYearByYearStats(team_id=wiz_id)
all_wiz_stats = wiz_stats.get_data_frames()[0]
all_wiz_stats.head(5)

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612764,Chicago,Packers,1961-62,80,18,62,0.225,0,5,...,0,0,0,1802,1954,0,0,0,8874,9
1,1610612764,Chicago,Zephyrs,1962-63,80,25,55,0.313,0,5,...,0,0,0,1773,2065,0,0,0,8795,8
2,1610612764,Baltimore,Bullets,1963-64,80,31,49,0.388,0,4,...,0,0,0,1423,2073,0,0,0,8948,5
3,1610612764,Baltimore,Bullets,1964-65,80,37,43,0.463,0,3,...,0,0,0,1676,2119,0,0,0,9087,2
4,1610612764,Baltimore,Bullets,1965-66,80,38,42,0.475,0,2,...,0,0,0,1890,2199,0,0,0,9465,2


In [26]:
all_wiz_stats.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [27]:
from nba_api.stats.endpoints import leaguedashteamstats

In [60]:
years = ['2010-11','2011-12','2012-13','2013-14','2014-15','2015-16','2016-17','2017-18','2018-19','2019-20']
years

['2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20']

In [61]:
all_scoring_stats = pd.DataFrame()
for year in years:
    scoring = leaguedashteamstats.LeagueDashTeamStats(season = year)
    scoring_stats = scoring.get_data_frames()[0]
    scoring_stats['SEASON'] = year
    all_scoring_stats = all_scoring_stats.append(scoring_stats)
all_scoring_stats['SEASON'].unique()

array(['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18', '2018-19', '2019-20'], dtype=object)

In [64]:
# Create CSV
all_scoring_stats.to_csv('team_stats.csv',index=False)

In [65]:
all_scoring_stats[all_scoring_stats['TEAM_NAME']=='Washington Wizards']

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS,SEASON
29,1610612764,Washington Wizards,82,23,59,0.280,3986.0,3048,6888,0.443,...,4,1,20,29,21,21,29,10,Washington Wizards,2010-11
29,1610612764,Washington Wizards,66,20,46,0.303,3173.0,2414,5475,0.441,...,13,2,7,26,23,22,26,10,Washington Wizards,2011-12
29,1610612764,Washington Wizards,82,29,53,0.354,3971.0,2910,6693,0.435,...,20,21,7,20,21,28,21,10,Washington Wizards,2012-13
29,1610612764,Washington Wizards,82,44,38,0.537,4011.0,3177,6920,0.459,...,11,15,7,13,23,16,15,10,Washington Wizards,2013-14
29,1610612764,Washington Wizards,82,46,36,0.561,3991.0,3139,6790,0.462,...,20,16,6,17,21,17,14,10,Washington Wizards,2014-15
29,1610612764,Washington Wizards,82,41,41,0.500,3951.0,3238,7033,0.460,...,8,26,6,21,16,9,17,10,Washington Wizards,2015-16
29,1610612764,Washington Wizards,82,49,33,0.598,3971.0,3388,7137,0.475,...,2,24,10,27,18,5,9,10,Washington Wizards,2016-17
29,1610612764,Washington Wizards,82,43,39,0.524,3971.0,3275,7018,0.467,...,10,22,13,25,15,13,15,10,Washington Wizards,2017-18
29,1610612764,Washington Wizards,82,32,50,0.390,3986.0,3456,7387,0.468,...,7,23,10,13,12,10,25,10,Washington Wizards,2018-19
29,1610612764,Washington Wizards,72,25,47,0.347,3471.0,2990,6544,0.457,...,10,22,20,30,4,8,25,10,Washington Wizards,2019-20


In [44]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [57]:
from nba_api.stats.endpoints import leaguedashplayerstats

In [66]:
all_season_stats = pd.DataFrame()
for year in years:
    all_stats = leaguedashplayerstats.LeagueDashPlayerStats(season = year)
    all_season_stats_x = all_stats.get_data_frames()[0]
    all_season_stats_x['SEASON'] = year
    all_season_stats = all_season_stats.append(all_season_stats_x)
all_season_stats['SEASON'].unique()

array(['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18', '2018-19', '2019-20'], dtype=object)

In [67]:
# Export to CSV
all_season_stats.to_csv('all_season_stats.csv',index=False)

In [ ]:
from nba_api.stats.endpoints import shotchartdetail

In [ ]:
beal_id = '203078'
beal_shotchart = shotchartdetail.ShotChartDetail(player_id = beal_id, team_id = wiz_id, season_nullable='2019-20',context_measure_simple='FGA')
beal_shotchart_stats = beal_shotchart.get_data_frames()[0]

In [ ]:
wiz_shotchart = shotchartdetail.ShotChartDetail(player_id = 0, team_id = wiz_id, season_nullable='2018-19',context_measure_simple='FGA')
wiz_shotchart_stats = wiz_shotchart.get_data_frames()[0]
wiz_shotchart_stats["SEASON"] = '2018-19'
wiz_shotchart_2 = shotchartdetail.ShotChartDetail(player_id = 0, team_id = wiz_id, season_nullable='2019-20',context_measure_simple='FGA')
wiz_shotchart_stats_2 = wiz_shotchart_2.get_data_frames()[0]
wiz_shotchart_stats_2["SEASON"] = '2019-20'
wiz_shot_stats = pd.DataFrame(wiz_shotchart_stats)
wiz_shot_stats = wiz_shot_stats.append(wiz_shotchart_stats_2)
wiz_shot_stats.head(5)

In [ ]:
#Print to CSV
#wiz_shot_stats.to_csv(r'C:\Users\jakes\Documents\DAEN\Fall 2020\AIT664\wiz_shot_stats.csv',index=False)

### Begin Creating Charts

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

#### Extra Possible Stats (unlikely to be needed)

In [ ]:
#playerdashboardbyshootingsplits
from nba_api.stats.endpoints import playerdashboardbyshootingsplits
def_stats = playerdashboardbyshootingsplits.PlayerDashboardByShootingSplits(player_id=beal_id, season='2019-20')
all_def_stats = pd.DataFrame()
for i in range(1,7):
    all_def_stats = all_def_stats.append(def_stats.get_data_frames()[i])
    i = i+1
all_def_stats.head(5)